In [1]:
import pandas as pd
import numpy as np
from skimage import io, color, feature
from skimage.transform import resize

import os
import pandas as pd
import numpy as np
from skimage import io, color, feature, transform
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from skimage import io, color, feature, transform
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [2]:
selected_images = pd.read_csv('../data/selected_images.csv')

def apply_mask(image, mask):
    if len(mask.shape) > 2:
        mask = color.rgb2gray(mask)
        mask = mask.astype(bool)

    if len(image.shape) > 2:
        for i in range(image.shape[2]):
            image[:,:,i] = np.where(mask, image[:,:,i], 0)
    else:
        image = np.where(mask, image, 0)
    return image

images = []
features = []

chunk_size = 50  
num_chunks = len(selected_images) // chunk_size + (len(selected_images) % chunk_size != 0)

min_feature_length = None

for chunk in range(num_chunks):

    chunk_features = []  

    for _, row in selected_images.iloc[chunk * chunk_size : (chunk + 1) * chunk_size].iterrows():
        image = io.imread(f"../data/images_resized/{row['img_id']}")
        mask = io.imread(f"../data/shanon_masks_total/{row['img_id']}")

        image = apply_mask(image, mask)

        image = resize(image, (image.shape[0]//2, image.shape[1]//2))

        feature_matrix = feature.multiscale_basic_features(image)
        feature_matrix = feature_matrix.reshape(-1)  

        if min_feature_length is None or feature_matrix.size < min_feature_length:
            min_feature_length = feature_matrix.size

        chunk_features.append(feature_matrix)

    chunk_features = [f[:min_feature_length] for f in chunk_features]

    chunk_features = np.array(chunk_features)

    np.save(f'chunk_{chunk}_features.npy', chunk_features)

c:\Users\jackh\AppData\Local\Programs\Python\Python310\lib\site-packages\skimage\feature\_basic_features.py:22: FutureWarning: Automatic detection of the color channel was deprecated in v0.19, and `channel_axis=None` will be the new default in v0.21. Set `channel_axis=-1` explicitly to silence this warning.
  gaussian_filtered = filters.gaussian(img, sigma, preserve_range=False)


In [3]:

all_features = []
for chunk in range(num_chunks):
    chunk_features = np.load(f'chunk_{chunk}_features.npy')
    all_features.append(chunk_features)

all_features = np.concatenate(all_features)

pca = PCA(n_components=7)
pca_features = pca.fit_transform(all_features)

diagnostics = np.where(selected_images['diagnostic'] == 'MEL', 1, 0)

patient_ids = selected_images['patient_id'].values
X_train_val, X_test, y_train_val, y_test = train_test_split(
    pca_features, diagnostics, test_size=0.2, random_state=42, stratify=diagnostics, shuffle=True)

X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.2, random_state=42, stratify=y_train_val, shuffle=True)

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

y_val_pred = knn.predict(X_val)

val_accuracy = accuracy_score(y_val, y_val_pred)
val_precision = precision_score(y_val, y_val_pred)
val_recall = recall_score(y_val, y_val_pred)

y_test_pred = knn.predict(X_test)

test_accuracy = accuracy_score(y_test, y_test_pred)
test_precision = precision_score(y_test, y_test_pred)
test_recall = recall_score(y_test, y_test_pred)

print("Validation Accuracy:", val_accuracy)
print("Test Accuracy:", test_accuracy)
print("Validation Precision:", val_precision)
print("Test Precision:", test_precision)
print("Validation Recall:", val_recall)
print("Test Recall:", test_recall)

Validation Accuracy: 0.6363636363636364
Test Accuracy: 0.5714285714285714
Validation Precision: 0.5
Test Precision: 0.42857142857142855
Validation Recall: 0.5
Test Recall: 0.2727272727272727


In [4]:
total_features = all_features.shape[0]
print("Total number of features:", total_features)

Total number of features: 138
